# UN resolution miner

In [1]:
import pandas as pd, urllib, requests, urllib2, json, numpy as np
datapath='E:/Dropbox/Public/datarepo/un/'

## Scrape and save resolution list for each session

In [ ]:
for i in range(1,72):
    print i,
    try:
        pd.read_html('https://www.un.org/depts/dhl/resguide/r'+str(i)+'_resolutions_table_en.htm',skiprows=1,header=0)[0].to_excel('res/'+str(i)+'en.xlsx')
    except:
        try:
            pd.read_html('https://www.un.org/depts/dhl/resguide/r'+str(i)+'_resolutions_table_eng.htm',skiprows=1,header=0)[0].to_excel('res/'+str(i)+'eng.xlsx')
        except:
            print 'ERROR',i

## Download individual resolutions

In [2]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(datapath+'pdf/') if isfile(join(datapath+'pdf/', f))]
links=[i[i.find('_')+1:i.rfind('_')].replace('-','/') for i in onlyfiles]
errors=[]

#### PDF

In [15]:
#save progress
file("links.json",'w').write(json.dumps(links))
file("errors.json",'w').write(json.dumps(list(set(errors))))
verbose=False
br=False
#parse new entries
for i in range(1,72):
    if verbose:print 'Session',i
    #load session file
    try: 
        df=pd.read_excel('res/'+str(i)+'en.xlsx')
    except:
        try:
            df=pd.read_excel('res/'+str(i)+'eng.xlsx')
        except:
            print 'ERROR',i
    #for all resolutions in that session
    for v in range(len(df[df.columns[0]].values)):
        if str(df[df.columns[0]].values[v]).lower()!='nan':
            j=df[df.columns[0]].values[v].replace(' ','')
            #formatting functions - UN filing conventions and taxonomies changed over time
            if ')' in j: j=j[:j.find(')')+1]
            for ii in range(5):
                if j[-1].isalpha():j=j[:-1]
                if j[-1] in {' ','-'}:j=j[:-1]

            if j not in links+errors:
                if verbose: print 'Resolution',j
                else: print j,
                try:
                    url='http://daccess-ods.un.org/'
                    link2='access.nsf/Get?Open&DS='+str(j)+'&Lang=E'
                    meta=requests.get(url+link2).content
                    #get temporary redirect link - 1st redirect
                    redir=meta[meta.find('URL')+5:meta.find('.html')]
                    if verbose:print url+redir+'.html'
                    meta2=requests.get(url+redir+'.html').content
                    #get pdf link - 2nd redirect
                    redir2=meta2[meta2.find('URL')+9:meta2.find('.pdf')]
                    #capture cookies
                    s = requests.Session()
                    cookielink=meta2[meta2.find('src')+5:meta2.find('Password=1234')+13]
                    cookiesopen = s.get(cookielink)
                    cookies=str(s.cookies)
                    fcookies=[[k[:k.find('=')],k[k.find('=')+1:k.find(' for ')]] for k in cookies[cookies.find('Cookie '):].split('Cookie ')[1:]]
                    if verbose:print 'http'+redir2+'.pdf'
                    #push cookies
                    opener = urllib2.build_opener()
                    for k in fcookies:
                        opener.addheaders.append(('Cookie', k[0]+'='+k[1]))
                    #open and save pdf
                    with open(datapath+"pdf/"+str(i)+"_"+str(j).replace('/','-')+"_"+redir2[redir2.rfind('/')+1:]+'.pdf', "wb") as code:
                        code.write(opener.open('http'+redir2+'.pdf?OpenElement').read())
                    code.close()
                    links.append(j)
                except:
                    errors.append(j)
                    #raise ValueError('Format error',j)
                    print 'error',j

In [16]:
file("links.json",'w').write(json.dumps(links))
file("errors.json",'w').write(json.dumps(list(set(errors))))

#### DOC

In [8]:
onlyfiles2 = [f for f in listdir(datapath+'doc/') if isfile(join(datapath+'doc/', f))]
links2=[i[i.find('_')+1:i.rfind('_')].replace('-','/') for i in onlyfiles2]
errors2=[]

In [13]:
#save progress
file("links2.json",'w').write(json.dumps(links2))
file("errors2.json",'w').write(json.dumps(list(set(errors2))))
verbose=False
br=False
#parse new entries
for i in range(48,72): #no word docuemnts before session 48
    if verbose:print 'Session',i
    #load session file
    try: 
        df=pd.read_excel('res/'+str(i)+'en.xlsx')
    except:
        try:
            df=pd.read_excel('res/'+str(i)+'eng.xlsx')
        except:
            print 'ERROR',i
    #for all resolutions in that session
    for v in range(len(df[df.columns[0]].values)):
        if str(df[df.columns[0]].values[v]).lower()!='nan':
            j=df[df.columns[0]].values[v].replace(' ','')
            #formatting functions - UN filing conventions and taxonomies changed over time
            if ')' in j: j=j[:j.find(')')+1]
            for ii in range(5):
                if j[-1].isalpha():j=j[:-1]
                if j[-1] in {' ','-'}:j=j[:-1]

            if j not in links2+errors2:
                if verbose: print 'Resolution',j
                else: print j,
                try:
                    url='http://daccess-ods.un.org/'
                    link2='access.nsf/Get?Open&DS='+str(j)+'&Lang=E'
                    meta=requests.get(url+link2).content
                    #get temporary redirect link - 1st redirect
                    redir=meta[meta.find('URL')+5:meta.find('.html')]
                    if verbose:print url+redir+'.html'
                    meta2=requests.get(url+redir+'.html').content
                    #get pdf link - 2nd redirect
                    redir2=meta2[meta2.find('URL')+9:meta2.find('.pdf')]
                    #capture cookies
                    s = requests.Session()
                    cookielink=meta2[meta2.find('src')+5:meta2.find('Password=1234')+13]
                    cookiesopen = s.get(cookielink)
                    cookies=str(s.cookies)
                    fcookies=[[k[:k.find('=')],k[k.find('=')+1:k.find(' for ')]] for k in cookies[cookies.find('Cookie '):].split('Cookie ')[1:]]
                    if verbose:print 'http'+redir2+'.pdf'
                    #push cookies
                    opener = urllib2.build_opener()
                    for k in fcookies:
                        opener.addheaders.append(('Cookie', k[0]+'='+k[1]))
                    #open and save doc
                    doclink='http://daccess-ods.un.org/access.nsf/GetFile?OpenAgent&DS='+str(j)+'&Lang=E&Type=DOC'
                    stream=opener.open(doclink).read()
                    if 'Error Handle in function SearchDS' not in stream:
                        with open(datapath+"doc/"+str(i)+"_"+str(j).replace('/','-')+"_"+redir2[redir2.rfind('/')+1:]+'.doc', "wb") as code:
                            code.write(stream)
                        code.close()
                    links2.append(j)
                except:
                    errors2.append(j)
                    #raise ValueError('Format error',j)
                    print 'error',j

In [14]:
file("links2.json",'w').write(json.dumps(links2))
file("errors2.json",'w').write(json.dumps(list(set(errors2))))

## Re-parse errors

In [47]:
genuinerrors=[]
fixedlinks=[]
verbose=True

In [48]:
for i in errors:
    j=i.replace('{','')
    for ii in range(5):
        if j[-1].isalpha():j=j[:-1]
        if j[-1] in {' ','-'}:j=j[:-1]
    if j not in genuinerrors+fixedlinks:
        url='http://daccess-ods.un.org/'
        link2='access.nsf/Get?Open&DS='+str(j)+'&Lang=E'
        meta=requests.get(url+link2).content
        if 'There is no document matching your request' in meta:
            genuinerrors.append(j)
            if verbose: print 'genuine error',url+link2
        elif 'This document is under EMBARGO' in meta:
            genuinerrors.append(j)
            if verbose: print 'genuine error',url+link2
        else:
            #get temporary redirect link - 1st redirect
            redir=meta[meta.find('URL')+5:meta.find('.html')]
            if verbose:print url+redir+'.html'
            meta2=requests.get(url+redir+'.html').content
            #get pdf link - 2nd redirect
            redir2=meta2[meta2.find('URL')+9:meta2.find('.pdf')]
            #capture cookies
            s = requests.Session()
            cookielink=meta2[meta2.find('src')+5:meta2.find('Password=1234')+13]
            cookiesopen = s.get(cookielink)
            cookies=str(s.cookies)
            fcookies=[[k[:k.find('=')],k[k.find('=')+1:k.find(' for ')]] for k in cookies[cookies.find('Cookie '):].split('Cookie ')[1:]]
            if verbose:print 'http'+redir2+'.pdf'
            #push cookies
            opener = urllib2.build_opener()
            for k in fcookies:
                opener.addheaders.append(('Cookie', k[0]+'='+k[1]))
            #open and save pdf
            with open(datapath+"pdf/"+j[j.find('RES')+4:][:2]+"_"+str(j).replace('/','-')+"_"+redir2[redir2.rfind('/')+1:]+'.pdf', "wb") as code:
                code.write(opener.open('http'+redir2+'.pdf?OpenElement').read())
            code.close()
            fixedlinks.append(j)

genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/35/220&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/37/235&Lang=E
http://daccess-ods.un.org/TMP/9349543.45226288.html
http://documents-dds-ny.un.org/doc/RESOLUTION/GEN/NR0/425/55/IMG/NR042555.pdf
http://daccess-ods.un.org/TMP/2584508.95547867.html
http://documents-dds-ny.un.org/doc/RESOLUTION/GEN/NR0/446/26/IMG/NR044626.pdf
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/38/183J*B108&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/47/54&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/48/230&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/50/125&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/53/203&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/54/250&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/54

In [62]:
genuinerrors2=[]
fixedlinks2=[]

In [63]:
for i in errors2:
    j=i.replace('{','')
    for ii in range(5):
        if j[-1].isalpha():j=j[:-1]
        if j[-1] in {' ','-'}:j=j[:-1]
    if j not in genuinerrors2+fixedlinks2:
        url='http://daccess-ods.un.org/'
        link2='access.nsf/Get?Open&DS='+str(j)+'&Lang=E'
        meta=requests.get(url+link2).content
        if 'There is no document matching your request' in meta:
            genuinerrors2.append(j)
            if verbose: print 'genuine error',url+link2
        elif 'This document is under EMBARGO' in meta:
            genuinerrors2.append(j)
            if verbose: print 'genuine error',url+link2
        else:
            #get temporary redirect link - 1st redirect
            redir=meta[meta.find('URL')+5:meta.find('.html')]
            if verbose:print url+redir+'.html'
            meta2=requests.get(url+redir+'.html').content
            #get pdf link - 2nd redirect
            redir2=meta2[meta2.find('URL')+9:meta2.find('.pdf')]
            #capture cookies
            s = requests.Session()
            cookielink=meta2[meta2.find('src')+5:meta2.find('Password=1234')+13]
            cookiesopen = s.get(cookielink)
            cookies=str(s.cookies)
            fcookies=[[k[:k.find('=')],k[k.find('=')+1:k.find(' for ')]] for k in cookies[cookies.find('Cookie '):].split('Cookie ')[1:]]
            if verbose:print 'http'+redir2+'.pdf'
            #push cookies
            opener = urllib2.build_opener()
            for k in fcookies:
                opener.addheaders.append(('Cookie', k[0]+'='+k[1]))
            if 'Error Handle in function SearchDS' not in stream:
                    with open(datapath+"doc/"+j[j.find('RES')+4:][:2]+"_"+str(j).replace('/','-')+"_"+redir2[redir2.rfind('/')+1:]+'.doc', "wb") as code:
                        code.write(stream)
                    code.close()
            fixedlinks2.append(j)

genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/48/230&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/50/125&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/53/203&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/54/250&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/54/247&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/54/237&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/54/96&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/55/239&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/56/254&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/56/220&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/57/293&Lang=E
genuine error http://daccess-ods.un.org/access.nsf/Get?Open&DS=A/RES/57/287&Lang=E
genui